**PROGETTO 3: TRAVEL TARGET - NOTEBOOK 2 - Pulizia e analisi dei dati**

Cioni Susanna, Tonsi Laura, Vignoli Filippo



# 📘 Introduzione

Questo secondo notebook contiene la **pulizia**, l’**organizzazione** e la **prima analisi esplorativa** dei dati ISTAT sui viaggi degli italiani.  
A partire dai tre dataset scelti (destinazione, mezzo di trasporto, tipo di alloggio), sono state realizzate aggregazioni e visualizzazioni che rispondono direttamente alle esigenze delle personae definite nel Notebook 1.

Il focus di questa fase è:
- Validare le user story con dati reali
- Individuare pattern regionali nei comportamenti di viaggio
- Costruire le basi analitiche per le raccomandazioni finali

Eseguendo la seguente cella di codice verrà richiesto di consentire l'accesso di Colab a Google Drive, è necessario accettare per poter proseguire, in quanto i dataset sono caricati su Google Drive.

In [ ]:
# --- Librerie ---
import pandas as pd
import os
from google.colab import drive

# --- Monta Google Drive ---
drive.mount('/content/drive')

# --- Percorso della cartella dei dataset ISTAT ---
project_folder = "/content/drive/MyDrive/Progetto 3/Dataset Istat"

# --- Percorsi completi ai file Excel salvati nella cartella ---
file_residenza_dest = os.path.join(project_folder, "Residenza_destinazione.xlsx")
file_mezzi = os.path.join(project_folder, "Residenza_mezzo di trasporto.xlsx")
file_alloggi = os.path.join(project_folder, "Residenza_tipo di alloggio.xlsx")

# --- Lettura dei file Excel (saltando le prime 4 righe come da specifica ISTAT) ---
df_residenza_dest = pd.read_excel(file_residenza_dest, skiprows=4, engine="openpyxl")
df_mezzi = pd.read_excel(file_mezzi, skiprows=4, engine="openpyxl")
df_alloggi = pd.read_excel(file_alloggi, skiprows=4, engine="openpyxl")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# --- Librerie ---
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- Utilizziamo i DataFrame già letti in memoria ---
residenza_dest_raw = df_residenza_dest
mezzi_raw = df_mezzi
alloggi_raw = df_alloggi

# --- Rinomina colonne chiave e Gestione delle celle unite (fill forward - ffill) ---
# --- Rinomina colonne chiave e gestisci celle unite (fill forward) ---
# Dataset 1: Residenza e Destinazione
residenza_dest_raw.rename(columns={residenza_dest_raw.columns[0]: "Residenza",
                                   residenza_dest_raw.columns[1]: "Destinazione"}, inplace=True)
residenza_dest_raw["Residenza"] = residenza_dest_raw["Residenza"].ffill()

# Dataset 2: Residenza e Mezzo di Trasporto
mezzi_raw.rename(columns={mezzi_raw.columns[0]: "Residenza",
                          mezzi_raw.columns[1]: "Mezzo"}, inplace=True)
mezzi_raw["Residenza"] = mezzi_raw["Residenza"].ffill()

# Dataset 3: Residenza e Tipo di Alloggio
alloggi_raw.rename(columns={alloggi_raw.columns[0]: "Residenza",
                            alloggi_raw.columns[1]: "Alloggio"}, inplace=True)
alloggi_raw["Residenza"] = alloggi_raw["Residenza"].ffill()

# --- Funzione di pulizia e melt (MODIFICATA PER FIX KeyError) ---
def melt_and_clean(df, id_vars, value_name, total_col_for_year):
    """
    Pulisce e trasforma un DataFrame selezionando una specifica colonna totale per l'anno.

    Args:
        df (pd.DataFrame): Il DataFrame da pulire e trasformare.
        id_vars (list): Lista di nomi di colonne da mantenere come identificatori.
        value_name (str): Nome della nuova colonna che conterrà i valori numerici.
        total_col_for_year (str): Il nome esatto della colonna che contiene il totale
                                  per l'anno (es. '2023 .2' o '2023 .4').

    Returns:
        pd.DataFrame: Il DataFrame pulito e in formato "long" con solo la colonna totale dell'anno.
    """
    df = df.copy()

    # Rimuovi le righe dove tutte le colonne id_vars sono NaN (dopo ffill)
    df = df.dropna(subset=id_vars, how='all')

    # Pulisci gli spazi extra e normalizza tutti i tipi di spazi nei nomi delle colonne
    # Questo è più robusto di solo .str.strip()
    df.columns = df.columns.str.replace(r'\s+', ' ', regex=True).str.strip()

    for col in id_vars:
        df[col] = df[col].astype(str).str.strip()

    # DEBUG: Stampa i nomi delle colonne disponibili all'interno della funzione
    print(f"\n--- DEBUG (inside melt_and_clean): Colonne disponibili: {df.columns.tolist()} ---")
    print(f"--- DEBUG (inside melt_and_clean): Colonna cercata: '{total_col_for_year}' ---")


    try:
        # Seleziona solo le colonne identificative e la specifica colonna totale per l'anno
        df_selected = df[id_vars + [total_col_for_year]].copy()
    except KeyError as e:
        # Cattura l'errore e fornisce un messaggio più utile
        print(f"\nERRORE: La colonna '{total_col_for_year}' non è stata trovata.")
        print(f"Colonne disponibili nel DataFrame in questo momento: {df.columns.tolist()}")
        raise e # Rilancia l'errore originale dopo aver stampato il debug

    df_selected.rename(columns={total_col_for_year: value_name}, inplace=True)

    # Aggiungi la colonna "Anno" con il valore "2023"
    df_selected["Anno"] = "2023"

    # Converte la colonna dei valori numerici al tipo numerico.
    df_selected[value_name] = pd.to_numeric(df_selected[value_name], errors="coerce")

    # Rimuovi le righe dove il valore numerico è diventato NaN
    df_selected.dropna(subset=[value_name], inplace=True)

    # Filtra le righe superflue dalla prima colonna identificativa (tipicamente "Residenza").
    df_selected = df_selected[~df_selected[id_vars[0]].str.lower().str.contains("tipo|territorio|totale")]

    return df_selected

# --- DEBUG: Stampa i nomi delle colonne prima di chiamare melt_and_clean ---
print("\n--- DEBUG: Nomi delle colonne in residenza_dest_raw (prima di melt_and_clean) ---")
print(residenza_dest_raw.columns.tolist())
print("\n--- DEBUG: Nomi delle colonne in mezzi_raw (prima di melt_and_clean) ---")
print(mezzi_raw.columns.tolist())
print("\n--- DEBUG: Nomi delle colonne in alloggi_raw (prima di melt_and_clean) ---")
print(alloggi_raw.columns.tolist())
print("\n" + "="*70 + "\n")


# --- Trasformazione in formato lungo per tutti i DataFrame (CHIAMATE MODIFICATE) ---
# Passiamo il nome della colonna che contiene il totale per il 2023 per ogni dataset
df_dest_long = melt_and_clean(residenza_dest_raw, ["Residenza", "Destinazione"], "Valore", "2023")
df_mezzi_long = melt_and_clean(mezzi_raw, ["Residenza", "Mezzo"], "Valore", "2023 .2")
df_alloggi_long = melt_and_clean(alloggi_raw, ["Residenza", "Alloggio"], "Valore", "2023 .2")


print("--- Dati Residenza e Destinazione (dopo melt e pulizia iniziale) ---")
print(df_dest_long.shape)
print(df_dest_long.head())

print("\n--- Dati Residenza e Mezzo (dopo melt e pulizia iniziale) ---")
print(df_mezzi_long.shape)
print(df_mezzi_long.head())

print("\n--- Dati Residenza e Alloggio (dopo melt e pulizia iniziale) ---")
print(df_alloggi_long.shape)
print(df_alloggi_long.head())


# --- Applicazione dei filtri specifici per ciascun dataset ---

# Dataset 1: Residenza e Destinazione
df_dest_long = df_dest_long[df_dest_long["Residenza"] != "Italia"]
df_dest_long = df_dest_long[~df_dest_long["Destinazione"].isin(["Italia", "Totale"])]

# Dataset 2: Residenza e Mezzo di Trasporto
df_mezzi_long = df_mezzi_long[df_mezzi_long["Residenza"] != "Italia"]
df_mezzi_long = df_mezzi_long[df_mezzi_long["Mezzo"] != "Tutte le voci"]

# Dataset 3: Residenza e Tipo di Alloggio
df_alloggi_long = df_alloggi_long[df_alloggi_long["Residenza"] != "Italia"]
df_alloggi_long = df_alloggi_long[df_alloggi_long["Alloggio"] != "Totale"]


# --- Stampa per verifica finale ---
print("\n--- Dati Residenza e Destinazione (dopo filtri finali) ---")
print(df_dest_long.shape)
print(df_dest_long.head())
print("Valori unici di 'Residenza':", df_dest_long["Residenza"].unique())
print("Valori unici di 'Destinazione':", df_dest_long["Destinazione"].unique())

print("\n--- Dati Residenza e Mezzo (dopo filtri finali) ---")
print(df_mezzi_long.shape)
print(df_mezzi_long.head())
print("Valori unici di 'Residenza':", df_mezzi_long["Residenza"].unique())
print("Valori unici di 'Mezzo':", df_mezzi_long["Mezzo"].unique())

print("\n--- Dati Residenza e Alloggio (dopo filtri finali) ---")
print(df_alloggi_long.shape)
print(df_alloggi_long.head())
print("Valori unici di 'Residenza':", df_alloggi_long["Residenza"].unique())
print("Valori unici di 'Alloggio':", df_alloggi_long["Alloggio"].unique())


# --- Salvataggio dei dataset puliti su Google Drive ---

# Crea la cartella di destinazione
save_folder = "/content/drive/MyDrive/Progetto 3/Dataset_puliti"
os.makedirs(save_folder, exist_ok=True)


# Salva i dataset puliti in formato CSV
# Salva i dataset puliti in formato CSV nella cartella corretta
df_dest_long.to_csv(os.path.join(save_folder, "dati_viaggi_destinazione_completi.csv"), index=False)
df_mezzi_long.to_csv(os.path.join(save_folder, "dati_viaggi_mezzi_completi.csv"), index=False)
df_alloggi_long.to_csv(os.path.join(save_folder, "dati_viaggi_alloggio_completi.csv"), index=False)


print("\nFile salvati correttamente su Drive nella cartella:", save_folder)


--- DEBUG: Nomi delle colonne in residenza_dest_raw (prima di melt_and_clean) ---
['Residenza', 'Destinazione', '2023  ', '2023  .1', '2023  .2', '2023  .3', '2023  .4']

--- DEBUG: Nomi delle colonne in mezzi_raw (prima di melt_and_clean) ---
['Residenza', 'Mezzo', '2023  ', '2023  .1', '2023  .2']

--- DEBUG: Nomi delle colonne in alloggi_raw (prima di melt_and_clean) ---
['Residenza', 'Alloggio', '2023  ', '2023  .1', '2023  .2']



--- DEBUG (inside melt_and_clean): Colonne disponibili: ['Residenza', 'Destinazione', '2023', '2023 .1', '2023 .2', '2023 .3', '2023 .4'] ---
--- DEBUG (inside melt_and_clean): Colonna cercata: '2023' ---

--- DEBUG (inside melt_and_clean): Colonne disponibili: ['Residenza', 'Mezzo', '2023', '2023 .1', '2023 .2'] ---
--- DEBUG (inside melt_and_clean): Colonna cercata: '2023 .2' ---

--- DEBUG (inside melt_and_clean): Colonne disponibili: ['Residenza', 'Alloggio', '2023', '2023 .1', '2023 .2'] ---
--- DEBUG (inside melt_and_clean): Colonna cercata: '2023

La pulizia dei dati è stata effettuata e i dataset puliti sono stati salvati su Google Drive.

# Verifica della corretta pulizia dei dataset

---



Verifichiamo i nomi di intestazioni di riga e di colonna, oltre alla natura dei dati contenuti (che la colonna "Valore" sia di tipo numerico - *int64 o float64* - e che le altre colonne siano di tipo *object*


In [ ]:
# --- Verifica dei Dataset Puliti ---

print("\n" + "="*50)
print("INIZIO VERIFICA DEI DATASET PULITI")
print("="*50 + "\n")

# --- Verifica df_dest_long (Residenza e Destinazione) ---
print(">>> VERIFICA: df_dest_long (Residenza e Destinazione) <<<")
print("\nDimensioni del DataFrame:", df_dest_long.shape)
print("\nPrime 5 righe:")
print(df_dest_long.head())
print("\nInformazioni sui tipi di dato:")
df_dest_long.info()
print("\nValori unici nella colonna 'Residenza':")
print(df_dest_long["Residenza"].unique())
print("\nValori unici nella colonna 'Destinazione':")
print(df_dest_long["Destinazione"].unique())
print("\nConteggio dei valori per 'Residenza':")
print(df_dest_long["Residenza"].value_counts())
print("\nConteggio dei valori per 'Destinazione':")
print(df_dest_long["Destinazione"].value_counts())
print("\n" + "-"*50 + "\n")


# --- Verifica df_mezzi_long (Residenza e Mezzo) ---
print(">>> VERIFICA: df_mezzi_long (Residenza e Mezzo di Trasporto) <<<")
print("\nDimensioni del DataFrame:", df_mezzi_long.shape)
print("\nPrime 5 righe:")
print(df_mezzi_long.head())
print("\nInformazioni sui tipi di dato:")
df_mezzi_long.info()
print("\nValori unici nella colonna 'Residenza':")
print(df_mezzi_long["Residenza"].unique())
print("\nValori unici nella colonna 'Mezzo':")
print(df_mezzi_long["Mezzo"].unique())
print("\nConteggio dei valori per 'Residenza':")
print(df_mezzi_long["Residenza"].value_counts())
print("\nConteggio dei valori per 'Mezzo':")
print(df_mezzi_long["Mezzo"].value_counts())
print("\n" + "-"*50 + "\n")


# --- Verifica df_alloggi_long (Residenza e Alloggio) ---
print(">>> VERIFICA: df_alloggi_long (Residenza e Tipo di Alloggio) <<<")
print("\nDimensioni del DataFrame:", df_alloggi_long.shape)
print("\nPrime 5 righe:")
print(df_alloggi_long.head())
print("\nInformazioni sui tipi di dato:")
df_alloggi_long.info()
print("\nValori unici nella colonna 'Residenza':")
print(df_alloggi_long["Residenza"].unique())
print("\nValori unici nella colonna 'Alloggio':")
print(df_alloggi_long["Alloggio"].unique())
print("\nConteggio dei valori per 'Residenza':")
print(df_alloggi_long["Residenza"].value_counts())
print("\nConteggio dei valori per 'Alloggio':")
print(df_alloggi_long["Alloggio"].value_counts())
print("\n" + "-"*50 + "\n")


print("\n" + "="*50)
print("FINE VERIFICA DEI DATASET PULITI")
print("="*50 + "\n")


INIZIO VERIFICA DEI DATASET PULITI

>>> VERIFICA: df_dest_long (Residenza e Destinazione) <<<

Dimensioni del DataFrame: (20, 4)

Prime 5 righe:
     Residenza  Destinazione  Valore  Anno
8   Nord-ovest  Paesi esteri   745.0  2023
10  Nord-ovest          Nord  3918.0  2023
11  Nord-ovest   Centro (IT)   854.0  2023
12  Nord-ovest    Sud, Isole   441.0  2023
14    Nord-est  Paesi esteri   628.0  2023

Informazioni sui tipi di dato:
<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 8 to 36
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Residenza     20 non-null     object 
 1   Destinazione  20 non-null     object 
 2   Valore        20 non-null     float64
 3   Anno          20 non-null     object 
dtypes: float64(1), object(3)
memory usage: 800.0+ bytes

Valori unici nella colonna 'Residenza':
['Nord-ovest' 'Nord-est' 'Centro' 'Sud' 'Isole']

Valori unici nella colonna 'Destinazione':
['Paesi esteri' 'N

Una volta verificata la correttezza della pulizia dei dataset, questi vengono salvati automaticamente su Drive e potranno essere utilizzati